In [1]:
import numpy as np
import scipy
import scipy.stats
import matplotlib.pyplot as plt
import growth_simulations as g
import weakref
import scipy.optimize

/home/felix/anaconda2/lib/python2.7/site-packages/matplotlib/__init__.py:1405: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)


In [2]:
%load_ext autoreload 

In [3]:
%autoreload 2

In [11]:
td = [np.log(2), np.log(2)]
td_std = [0.15, 0.15]
# td = [1.0, 1.0]
# td_std = [0.1, 0.1]

# this script uses a simple adder model

par1 = {'nstep': 1000, 'dt':0.01, 'td': td, 'td_std': td_std, 'v_init':1.0, 'modeltype':1, 'delta':1.0, 
        'lambda_std':None}
# par1['lambda'] = scipy.optimize.fsolve(g.fn_1, x0=np.log(2.0) / np.mean(td), args=(par1['td'][0], par1['td'][1]))[0]
# temp = 0.5/(np.exp(par1['lambda']*par1['td'][1])-1)
# par1['r'] = temp/(1-temp)
par1['r']=0.5
par1['lambda']=1.0

In [12]:
init_pop = g.starting_popn(par1)
c,obs = g.discr_time_1(par1, init_pop)

In [13]:
temp2 = scipy.stats.linregress(obs[1][400:], np.log(obs[7][400:]))
print temp2[0]

0.999913401763


In [7]:
for i0 in range(2):
    print 'celltype =', i0
    temp1 = [obj.t_grow for obj in c[10**4:] if obj.celltype==i0]
    temp2 = [obj.parent.t_grow for obj in c[10**4:] if obj.celltype==i0]
    vals = scipy.stats.linregress(temp1, temp2)
    print vals[2]*np.absolute(vals[2]), 'parent child'
    if i0==1:
        temp3 = [obj.parent_current.t_grow for obj in c[10**4:] if obj.celltype == i0]
        vals = scipy.stats.linregress(temp1, temp3)
        print vals[2]*np.absolute(vals[2]), 'mother daughter same generation'
        vals = scipy.stats.linregress(temp2, temp3)
        print vals[2]*np.absolute(vals[2]), 'mother daughter diff generation'

celltype = 0
0.102713945227 parent child
celltype = 1
0.109175927417 parent child
0.134166006458 mother daughter same generation
0.102713945227 mother daughter diff generation


In [77]:
td = [1.858, 3.353]
# td_std = [0.4727, 0.9214]
# td = [1.0, 1.0]
td_std = [0.001, 0.001]

# this script uses a simple adder model


par1 = {'nstep': 1000, 'dt':0.01, 'td': td, 'td_std': td_std, 'v_init':1.0, 'modeltype':1, 'delta':1.0, 
        'lambda_std':0.2}

# defining r and lambda to approximate the same average division times for daughters and mothers.

par1['lambda'] = scipy.optimize.fsolve(g.fn_1, x0=np.log(2.0) / np.mean(td), args=(par1['td'][0], par1['td'][1]))[0]
temp = 0.5/(np.exp(par1['lambda']*par1['td'][1])-1)
par1['r'] = temp/(1-temp)
# par1['r'] = 0.6
# par1['lambda'] = 1.0

In [78]:
init_pop = g.starting_popn(par1)
c,obs = g.discr_time_1(par1, init_pop)

In [79]:
temp2 = scipy.stats.linregress(obs[1][400:], np.log(obs[7][400:]))
print temp2[0]/par1['lambda']
temp2 = scipy.stats.linregress(obs[1][400:], np.log(obs[4][400:]))
print temp2[0]/par1['lambda']

0.972047886052
0.972370898401


In [22]:
temp2 = scipy.stats.linregress(obs[1][400:], np.log(obs[7][400:]))
print temp2[0]/par1['lambda']

0.903258027613


In [76]:
for i0 in range(2):
    print 'celltype =', i0
    temp1 = [obj.t_grow for obj in c[10**4:] if obj.celltype==i0]
    print np.mean(temp1)
    temp2 = [obj.parent.t_grow for obj in c[10**4:] if obj.celltype==i0]
    vals = scipy.stats.linregress(temp1, temp2)
    print vals[2]*np.absolute(vals[2]), 'parent child', scipy.stats.pearsonr(temp1, temp2)[0]
    if i0==1:
        temp3 = [obj.parent_current.t_grow for obj in c[10**4:] if obj.celltype == i0]
        vals = scipy.stats.linregress(temp1, temp3)
        print vals[2]*np.absolute(vals[2]), 'mother daughter same generation', scipy.stats.pearsonr(temp1, temp3)[0]
        vals = scipy.stats.linregress(temp2, temp3)
        print vals[2]*np.absolute(vals[2]), 'mother daughter diff generation', scipy.stats.pearsonr(temp2, temp3)[0]

celltype = 0
1.9357189171
0.0100091144743 parent child 0.100045561992
celltype = 1
3.46974048927
0.00776618564654 parent child 0.088125964656
0.117158704376 mother daughter same generation 0.342284537156
0.0100091144743 mother daughter diff generation 0.100045561992
